In [64]:
# Copy from https://stackoverflow.com/questions/30740046/calculate-distance-to-nearest-feature-with-geopandas
%matplotlib inline
import matplotlib.pyplot as plt
import shapely.geometry as geom
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import gdal 
from tqdm import tqdm

def pixel2coord(col, row):
    """Returns global coordinates to pixel center using base-0 raster index"""
    xp = a * col + b * row + a * 0.5 + b * 0.5 + c
    yp = d * col + e * row + d * 0.5 + e * 0.5 + f
    return(xp, yp)

raster = './img/ptg_mussles.tif'
block_x = 100 # ist gleich block_y, nur quadratische Ausschnitte
band_number = 1 # band number of tif to use
column_names_of_csv = ['Latitude', 'Longitude', 'filename']  # File Format of Nadir file

#read Raster
ds = gdal.Open(raster)  #open tif
band = ds.GetRasterBand(band_number)  #read first band assuming to include the BS data
# unravel GDAL affine transform parameters to get back coordinates
c, a, b, f, d, e = ds.GetGeoTransform()

max_width,max_height=ds.RasterXSize,ds.RasterYSize
x_runs=list(range(0,max_width - block_x, block_x))
y_runs=list(range(0,max_height - block_x, block_x))

for width_index in tqdm(x_runs):
    for height_index in y_runs:
        grey_array = band.ReadAsArray(width_index, height_index, block_x, block_x)  #the subset image with the specified channel
        # damit habe ich band 1. jetzt muss die entfernung jedes pixels (bzw seiner coordinaten) zu einer linie dazu
        # hier ist der offset noch nicht berücksichtigt
        pixel_coordinates = np.array([pixel2coord(xi[0][0], xi[0][1]) for xi in np.ndenumerate(grey_array)])

        #make geopandas series
        gpd_pixel_coordinates = gpd.GeoSeries([geom.Point(row[0], row[1]) for row in pixel_coordinates])

        # Read Nadir
        temp = pd.read_csv('test', sep = ',', names = column_names_of_csv)
        geometry = [Point(xy) for xy in zip(temp.Longitude, temp.Latitude)]
        df_lines = gpd.GeoDataFrame(temp, geometry=geometry)
        df_lines = df_lines.groupby(['filename'])['geometry'].apply(lambda x: LineString(x.tolist()) if x.size > 1 else x.tolist())


        # Get distance to Nadir for each Pixel
        min_dist = np.empty(len(gpd_pixel_coordinates))
        for i, point in enumerate(gpd_pixel_coordinates):
            min_dist[i] = np.min([point.distance(line) for line in df_lines])

        # auf 0-255 skalieren
        min_dist = ((min_dist - np.min(min_dist)) / (np.max(min_dist) - np.min(min_dist))) * 255 
        min_dist = np.round(min_dist).astype(int)

        # add min dist to grey_image as band
        min_dist = min_dist.reshape(block_x,block_x) # reshape to block_x, block_x (nur Quadrate)

        merge = np.dstack((grey_array, min_dist))

        #add texture or second bs channel? -> for now using the same channel twice
        merge = np.dstack((merge, grey_array))

        # plot
        #plt.imshow(grey_array, cmap='gray')
        #plt.colorbar()
        #plt.show()
        
        # pass to retinanet




  0%|          | 0/38 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [63]:
print(df_lines)
print(pixel_coordinates)

filename
xx    LINESTRING (10 54, 11 55, 11 56)
Name: geometry, dtype: object
[[ 746017.10600667 6056186.68281129]
 [ 746017.10600667 6056186.43283388]
 [ 746017.10600667 6056186.18285648]
 ...
 [ 746041.85732769 6056162.43500254]
 [ 746041.85732769 6056162.18502513]
 [ 746041.85732769 6056161.93504772]]


In [ ]:


)





In [ ]:
data = myfile.ReadAsArray(x_offset, y_offset, x_size, y_size)